In [69]:
import sys
import os
import json
import pandas
import numpy
import optparse
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from collections import OrderedDict
from sklearn.model_selection import train_test_split 
from sklearn.neighbors import KNeighborsClassifier

In [70]:
requests_data = pandas.read_csv('Datasets/output_http_csic_2010_full.csv')
requests_data = requests_data.head(5)

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [71]:
X = requests_data.iloc[:, :17]
X

,index,method,url,protocol,userAgent,pragma,cacheControl,accept,acceptEncoding,acceptCharset,acceptLanguage,host,connection,contentLength,contentType,cookie,payload
0,0,GET,http://localhost:8080/tienda1/publico/anadir.jsp,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,close,NaN,NaN,JSESSIONID=B92A8B48B9008CD29F622A994E0F650D,id=2
1,0,GET,http://localhost:8080/tienda1/publico/anadir.jsp,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,close,NaN,NaN,JSESSIONID=B92A8B48B9008CD29F622A994E0F650D,nombre=Jam%F3n+Ib%E9rico
2,0,GET,http://localhost:8080/tienda1/publico/anadir.jsp,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,close,NaN,NaN,JSESSIONID=B92A8B48B9008CD29F622A994E0F650D,precio=85
3,0,GET,http://localhost:8080/tienda1/publico/anadir.jsp,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,close,NaN,NaN,JSESSIONID=B92A8B48B9008CD29F622A994E0F650D,cantidad=%27%3B+DROP+TABLE+usuarios%3B+SELECT+...
4,0,GET,http://localhost:8080/tienda1/publico/anadir.jsp,HTTP/1.1,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,no-cache,no-cache,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,localhost:8080,close,NaN,NaN,JSESSIONID=B92A8B48B9008CD29F622A994E0F650D,B1=A%F1adir+al+carrito


In [72]:
Y = requests_data.iloc[:, 17]
Y

0    anom
1    anom
2    anom
3    anom
4    anom
Name: label, dtype: object

In [75]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

X_encoded=le.fit_transform(numpy.array(X))
X_encoded

ValueError: bad input shape (5, 17)

In [53]:
# Create dictionary
X = X.fillna( 'NA' )
X_dic = X.to_dict( orient = 'records' )

In [54]:
X_dic

[{'accept': 'text/xml,application/xml,application/xhtml+xml,text/html;q=0.9,text/plain;q=0.8,image/png,*/*;q=0.5',
  'acceptCharset': 'utf-8, utf-8;q=0.5, *;q=0.5',
  'acceptEncoding': 'x-gzip, x-deflate, gzip, deflate',
  'acceptLanguage': 'en',
  'cacheControl': 'no-cache',
  'connection': 'close',
  'contentLength': 'NA',
  'contentType': 'NA',
  'cookie': 'JSESSIONID=B92A8B48B9008CD29F622A994E0F650D',
  'host': 'localhost:8080',
  'index': 0,
  'method': 'GET',
  'payload': 'id=2',
  'pragma': 'no-cache',
  'protocol': 'HTTP/1.1',
  'url': 'http://localhost:8080/tienda1/publico/anadir.jsp',
  'userAgent': 'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.8 (like Gecko)'},
 {'accept': 'text/xml,application/xml,application/xhtml+xml,text/html;q=0.9,text/plain;q=0.8,image/png,*/*;q=0.5',
  'acceptCharset': 'utf-8, utf-8;q=0.5, *;q=0.5',
  'acceptEncoding': 'x-gzip, x-deflate, gzip, deflate',
  'acceptLanguage': 'en',
  'cacheControl': 'no-cache',
  'connection': 'close',
  'co

In [55]:
from sklearn.feature_extraction import DictVectorizer as DV

vectorizer = DV( sparse = False )
X = vectorizer.fit_transform(requests_dic)

In [56]:
X

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1.,
        0., 0., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0.,
        1., 0., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0.,
        0., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0.,
        0., 0., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0.,
        0., 0., 1., 1., 1., 1.]])

In [57]:
Y_dic = Y.to_dict()

vectorizer = DV( sparse = False )
Y = vectorizer.fit_transform(Y_dic)
Y

array([[1., 1., 1., 1., 1.]])

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=23)

ValueError: Found input variables with inconsistent numbers of samples: [5, 1]

In [65]:
# instantiate learning model (k = 3)
knn = KNeighborsClassifier(n_neighbors=3)

# fitting the model
knn.fit(X_train, y_train)

# predict the response
pred = knn.predict(X_test)

ValueError: Expected 2D array, got 1D array instead:
array=[{'accept': 'text/xml,application/xml,application/xhtml+xml,text/html;q=0.9,text/plain;q=0.8,image/png,*/*;q=0.5', 'contentLength': 'NA', 'protocol': 'HTTP/1.1', 'method': 'GET', 'contentType': 'NA', 'connection': 'close', 'index': 0, 'cookie': 'JSESSIONID=B92A8B48B9008CD29F622A994E0F650D', 'host': 'localhost:8080', 'acceptEncoding': 'x-gzip, x-deflate, gzip, deflate', 'acceptLanguage': 'en', 'pragma': 'no-cache', 'url': 'http://localhost:8080/tienda1/publico/anadir.jsp', 'payload': 'id=2', 'userAgent': 'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.8 (like Gecko)', 'acceptCharset': 'utf-8, utf-8;q=0.5, *;q=0.5', 'cacheControl': 'no-cache'}
 {'accept': 'text/xml,application/xml,application/xhtml+xml,text/html;q=0.9,text/plain;q=0.8,image/png,*/*;q=0.5', 'contentLength': 'NA', 'protocol': 'HTTP/1.1', 'method': 'GET', 'contentType': 'NA', 'connection': 'close', 'index': 0, 'cookie': 'JSESSIONID=B92A8B48B9008CD29F622A99